In [ ]:
import sys
sys.path.append('..\\')
import os
import re
import numpy as np
import gensim
import nltk
from gensim.models.word2vec import Word2Vec
from gensim.models.fasttext import FastText
from scripts.epoch_logger import EpochLogger
from scripts.predict_from_context import predict_output_word
from scripts.preprocess_data import load_data
from scripts.embeddingCorrector import EmbeddingCorrector


## Training

In [ ]:
# DATA_FOLDER = "..\\data\\"
# train_path = os.path.join(DATA_FOLDER, "train.txt")
# with open(train_path) as file:
#     text = file.read()

In [ ]:
# text = re.sub(r"\b(?:[a-z.]*[A-Z][a-z.]*){2,}", "", text)
# data = [nltk.word_tokenize(re.sub(r"[^a-z]+", " ", sentence.lower())) for sentence in nltk.sent_tokenize(text)]

In [ ]:
path = "..\\data\\pubmed-rct-master\\PubMed_200k_RCT\\train.txt"
data, _ = load_data(path)

### FastText

In [ ]:
epoch_logger = EpochLogger()
fast_text_model = FastText(sg=1, min_n=2, max_n=4, min_count = 5, size=300, workers=8, callbacks=(epoch_logger,))

# build the vocabulary
fast_text_model.build_vocab(data)

In [ ]:
print(fast_text_model)

In [ ]:
# train the model
fast_text_model.train(data, total_examples=fast_text_model.corpus_count, epochs=20, callbacks=(epoch_logger,))

In [ ]:
fast_text_model.save('..\\models\\fasttext200k_2_4_5_300\\fasttext200k.model')

### Word2Vec

In [ ]:
word2vec_model = Word2Vec(size=300, workers=12)
word2vec_model.build_vocab(data)

In [ ]:
epoch_logger = EpochLogger()
word2vec_model.train(data, total_examples=word2vec_model.corpus_count, epochs=20, callbacks=[epoch_logger])

In [ ]:
word2vec_model.save("..\\models\\word2vec200k\\word2vec200k.model")

## Testing

In [ ]:
# test_sent = "The level of postoperative pain and nausea was scored in the postanesthesia care unit ( PACU ) during the first postoperative hour , as well as at 2 , 4 , 8 and 24h postoperatively ."
# words = nltk.word_tokenize(re.sub(r"[^a-z]+", " ", test_sent.lower()))
# words

In [ ]:
fast_text_model = FastText.load("..\\models\\fasttext200k\\fasttext200k.model")
word2vec_model = Word2Vec.load("..\\models\\word2vec200k\\word2vec200k.model")

In [ ]:
path = "..\\data\\pubmed-rct-master\\PubMed_200k_RCT\\test.txt"
source, target = load_data(path)

In [ ]:
def score(ec, source, target, x = 1000):
    scr = 0
    amt = 0
    edited = ec.correct_text(source[:x])
    for i in range(len(source[:x])):
        if ' '.join(source[i]) != ' '.join(target[i]):
            amt += 1
            scr += 1 if ' '.join(edited[i]) == ' '.join(target[i]) else 0
#             if not all([(source[i][j]==target[i][j] and edited[i][j] == target[i][j]) or (source[i][j]!=target[i][j]) for j in range(len(source[i]))]):
#                 print("Source -> ", ' '.join(source[i]))
#                 print("Edited -> ", ' '.join(edited[i]))
#                 print("Target -> ", ' '.join(target[i]))

    return scr / amt

In [ ]:
ec = EmbeddingCorrector(fast_text_model)
score(ec, source, target, x = len(source))

In [ ]:
ec = EmbeddingCorrector(word2vec_model)
score(ec, source, target, x = len(source))